In [396]:
%matplotlib qt5
import os, sys, fnmatch
import cv2, time
import numpy as np
import argparse, pprint
import matplotlib

from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

In [397]:
def grab_dir_images(_dir, patterns = ['*png','*jpg'],verbose=False):
    found = []
    for root, dirs, files in os.walk(_dir):
        for pat in patterns:
            for file in files:
                if fnmatch.fnmatch(file, pat):
                    found.append(os.path.join(root, file))
    
    if(verbose): print(found)
    return found

In [398]:
def plot_image(img,figNum=None):
    if(figNum == None): plt.figure()
    else: plt.figure(figNum)
        
    plt.imshow(img)
    plt.subplots_adjust(wspace=0.0,hspace=0.0,left=0.0,right=1.0,top=1.0, bottom=0.0)
    plt.show()

In [399]:
def uvMapping(_img, get_overlay=True,verbose=False):
    overlay = None
    h,w = _img.shape
    histRange = (0,256)
    histSz = np.max(_img) + 1
    if(verbose): print("[UV Mapping] Input Image Size: (%d, %d)" % (h,w))

    umap = np.zeros((histSz,w,1), dtype=np.uint8)
    vmap = np.zeros((h,histSz,1), dtype=np.uint8)

    for i in range(0,h):
        vscan = _img[i,:]
        vrow = cv2.calcHist([vscan],[0],None,[histSz],histRange)
        if(verbose): print("\t[V Mapping] Scan [%d] (%s) ---- Scan Histogram (%s)" % (i,', '.join(map(str, vscan.shape)), ', '.join(map(str, vrow.shape))))
        vmap[i,:] = vrow

    for i in range(0,w):
        uscan = _img[:,i]
        urow = cv2.calcHist([uscan],[0],None,[histSz],histRange)
        if(verbose): print("\t[U Mapping] Scan[%d] (%s) ---- Scan Histogram (%s)" % (i,', '.join(map(str, uscan.shape)), ', '.join(map(str, urow.shape))))
        umap[:,i] = urow

    umap = np.reshape(umap,(histSz,w))
    vmap = np.reshape(vmap,(h,histSz))

    if(get_overlay):
        blank = np.ones((umap.shape[0],vmap.shape[1]),np.uint8)*255
        pt1 = np.concatenate((_img, vmap), axis=1)
        pt2 = np.concatenate((umap,blank),axis=1)
        overlay = np.concatenate((pt1,pt2),axis=0)
        overlay = cv2.cvtColor(overlay,cv2.COLOR_GRAY2BGR)

    if(verbose):
        print("\t[UV Mapping] U Map = (%s) ----- V Map = (%s)" % (', '.join(map(str, umap.shape)),', '.join(map(str, vmap.shape)) ))
    return umap,vmap,overlay

In [400]:
def construct_helper_img(_imgs,cspace=cv2.COLOR_GRAY2BGR):
    print(_imgs[0].shape)
    n,m = _imgs[0].shape[0], _imgs[0].shape[1]
    print(n,m)
    
    bborder = np.ones((5,m,3),dtype=np.uint8)
    sborder = np.ones((n,5,3),dtype=np.uint8)
    cborder = np.ones((5,5,3),dtype=np.uint8)
    bborder[np.where((bborder==[1,1,1]).all(axis=2))] = [255,0,255]
    sborder[np.where((sborder==[1,1,1]).all(axis=2))] = [255,0,255]
    cborder[np.where((cborder==[1,1,1]).all(axis=2))] = [255,0,255]
    
    imgs = []
    for img in _imgs:
        im = None
        try:
            im = cv2.cvtColor(img,cspace)
        except:
            im = img
        imgs.append(im)
    
    helper_img_t = np.concatenate(
        (
            np.concatenate((imgs[0],sborder), axis=1),
            np.concatenate((imgs[1],sborder), axis=1),
            np.concatenate((imgs[2],sborder), axis=1)
        ), axis=1
    )
    
    helper_img_m = np.concatenate(
        (
            np.concatenate((bborder,cborder), axis=1),
            np.concatenate((bborder,cborder), axis=1),
            np.concatenate((bborder,cborder), axis=1)
        ), axis=1
    )
    
    helper_img_b = np.concatenate(
        (
            np.concatenate((imgs[3],sborder), axis=1),
            np.concatenate((imgs[4],sborder), axis=1),
            np.concatenate((imgs[5],sborder), axis=1)
        ), axis=1
    )
    
    helper_img = np.concatenate((helper_img_t,helper_img_m,helper_img_b), axis=0)
    return helper_img

In [401]:
def abstract_horizontals(_img,threshold, show_contours=True,verbose=False):
    try: img = cv2.cvtColor(_img,cv2.COLOR_BGR2GRAY)
    except: img = _img
    n,m = img.shape[0], img.shape[1]
    print("Countours:", img.shape)
    _, contours, hierarchy = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    
    filtered_cnts = [cnt for cnt in contours if cv2.contourArea(cnt) >= threshold]
    filtered_areas = [cv2.contourArea(cnt) for cnt in filtered_cnts]
    
    # Find horizontal limits
    limits = []; ellipses = []; pts = []; centers = []
    count = 0
    for cnt in filtered_cnts:
        ellipse = cv2.fitEllipse(cnt)
        cx, cy = np.int32(ellipse[0])
        h, w = np.int32(ellipse[1])
        dx,dy = w/2,h/2
        
        if(verbose):
            print("Center: (%d, %d)" % (cx,cy))
            print("H, W: (%d, %d)" % (dx,dy))

        xr = cx + dx; xl = cx - dx
        yt = cy - dy; yb = cy + dy        
        ptL = (xl,cy); ptR = (xr,cy)
        
        lim = np.array([[xl,xr],[yt,yb]])
        limits.append(lim)
        
        pts.append([ptL, ptR])
        ellipses.append(ellipse)
        centers.append((cx,cy))        
        count+=1
        
    if(verbose):
        print("Raw Contour Areas:",areas)
        print("Filtered Contour Areas:",filtered_areas)
    
    
    if(show_contours):
        helper1 = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
        helper2 = np.copy(helper1)
        helper_in = np.copy(helper1)
        for i in range(count):
            cnt = filtered_cnts[i]
            cv2.drawContours(helper1, [cnt], 0, (255,255,255), 2)
            cv2.ellipse(helper2,ellipses[i],(0,255,0),2)
            cv2.circle(helper2,centers[i],2,(255,0,255), 5)
            cv2.circle(helper2,pts[i][0],2,(0,255,255), 5)
            cv2.circle(helper2,pts[i][1],2,(0,255,255), 5)
    
        sborder = np.ones((n,5,3),dtype=np.uint8)
        sborder[np.where((sborder==[1,1,1]).all(axis=2))] = [255,0,255]
        helper = np.concatenate((
            np.concatenate((helper_in,sborder), axis=1),
            np.concatenate((helper1,sborder), axis=1),
            np.concatenate((helper2,sborder), axis=1)
        ), axis=1)
        plot_image(helper,3)
        
    return filtered_cnts, limits, ellipses, pts, centers

In [402]:
flag_new_img = False
last_img = None
input_img = None
umap = None
vmap = None
overlay = None
helper_img = None
filtered = None
cpy1 = None
cpy2 = None
pre_filter = None

prev_e1,prev_e2,prev_greyThresh = 0, 0, 0
prev_map = False

found_imgs = grab_dir_images(homeDir)

In [443]:
def line_finder(
    _img="test/test_disparity.png", method = 1, line_input_method = 0, e1 = 2, e2 = 5,
    ang = 90, rho = 1, minLineLength = 11, maxLineGap = 11, houghThresh = 50, greyThresh = 11,
    cnt_thresh = 500.0, show_helpers = True, use_umap = True, flip_thresh_bin = False
    ):
    
    global filtered, pre_filter, umap, vmap, cpy1, cpy2, overlay, helper_img, prev_img
    global flag_new_img, last_img, prev_e1, prev_e2, prev_greyThresh, prev_map
    
    """
    Parse Input Variables to Create string desscriptors for easier control variable copying
    """
    if(line_input_method is 1): filter_meth = "Composite Filtering -> Blurring"
    elif(line_input_method is 2): filter_meth = "Composite Filtering -> Canny Edges"
    else: filter_meth = "Basic Thresholding"
        
    if(method is 0): line_meth = "Standard Hough Transform"
    else: line_meth = "Probablistic Hough Transform"
    
    if(use_umap): map_space = "U-Map"
    else: map_space = "V-Map"
    
    """
    Check control variable arguments for changes to reduce repeating unnecessary calculations
    """
    
    # Check if we are testing for a different image
    if(last_img == _img): flag_new_img = False
    else: flag_new_img = True

    # Check if we need to perform image filtering again
    if((prev_map is not use_umap) or (flag_new_img) or
       (prev_greyThresh is not greyThresh) or 
       (prev_e1 is not e1) or 
       (prev_e2 is not e2)
    ):
        flag_needs_filtering = True
        print("Images need to be filtered again...")
    else:
        flag_needs_filtering = True
        print("Skipping image filtering...")
    
    last_img, prev_e1, prev_e2, prev_greyThresh, prev_map = _img, e1, e2, greyThresh, use_umap
    
    print(
    """
    Inputs:  (image = %s)
    ------
    
    \t* Mapping Space       : %s
    \t* Line Finding Method : %s
    \t* Filtering Used      : %s
    \t* Kernel Size         : (%d, %d)
    \t* Rho, Angle (deg)    :  %d, %d
    \t* Min Line Length     :  %d
    \t* Max Line Gap        :  %d
    \t* Grey Thresholding   :  %d
    \t* Hough Threshold     :  %d
    \t* Contour Threshold   :  %d
    """ % (
        _img,map_space,line_meth,filter_meth,e1,e2,rho,ang,minLineLength,maxLineGap,greyThresh,houghThresh,cnt_thresh
    ))
    
    # Convert Angle to radians for calculations
    ang = ang * np.pi/180
    img = cv2.imread(_img,cv2.IMREAD_GRAYSCALE)
    if(flag_new_img): 
        print("Mapping new image into UV Map")
        umap,vmap, overlay = uvMapping(img)

    if(use_umap): tmpIn = np.copy(umap)
    else: tmpIn = np.copy(vmap)

    tmp = cv2.cvtColor(tmpIn,cv2.COLOR_GRAY2BGR)
    n,m,_ = tmp.shape
    filtered = np.zeros((n,m,3),dtype=np.uint8)

    if(flag_needs_filtering):
        if(flip_thresh_bin): masking = cv2.THRESH_BINARY_INV
        else: masking = cv2.THRESH_BINARY
            
        kernel = np.ones((e1,e2),np.uint8)
        kernel2 = np.ones((4,4),np.uint8)

        ret, grey = cv2.threshold(tmpIn,greyThresh,255,masking)
        dilation = cv2.dilate(grey,kernel,iterations = 1)
        blur = cv2.GaussianBlur(dilation,(5,5),0)
        closing = cv2.morphologyEx(grey,cv2.MORPH_CLOSE,kernel, iterations = 2)

        ret, grey_thresh = cv2.threshold(grey,greyThresh,255,masking)
        ret, close_thresh = cv2.threshold(closing,greyThresh,255,masking)
        canny = cv2.Canny(blur,25,200,apertureSize = 3)

        helper_imgs = [tmp,grey,dilation,blur,closing,canny]
        helper_img = construct_helper_img(helper_imgs)
        
    if(line_input_method is 1): hlines = blur
    elif(line_input_method is 2): hlines = canny
    else: hlines = grey
#     cv2.imwrite('input-lines.png',hlines)
    try:
        if(method==0):
            lines = cv2.HoughLines(hlines,rho,ang,houghThresh)
            count = 0
            for rho,theta in lines[0]:
                count+=1
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))

                cv2.line(tmp,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.line(filtered,(x1,y1),(x2,y2),(255,255,255),2)
        else:
            lines = cv2.HoughLinesP(hlines,rho,ang,houghThresh,minLineLength,maxLineGap)
            for x in range(0, len(lines)):
                for x1,y1,x2,y2 in lines[x]:
                    cv2.line(tmp,(x1,y1),(x2,y2),(0,255,0),2)
                    cv2.line(filtered,(x1,y1),(x2,y2),(255,255,255),2)
    except:
        print("Couldn't Find Hough Lines!!!")
        pass
    
    if(flag_new_img): plot_image(overlay,0)    
    if(flag_needs_filtering and show_helpers): plot_image(helper_img,1)
    try: plot_image(filtered,2)
    except: pass

    # CONTOUR SECTIION    
    pre_filter = cv2.GaussianBlur(filtered,(5,5),0)
#     try: cv2.imwrite('pre-filtered-lines.png',pre_filter)
#     except: pass
    pre_filter = cv2.cvtColor(pre_filter,cv2.COLOR_BGR2GRAY)
    abstract_horizontals(pre_filter,cnt_thresh)

    tmpI = np.copy(img)
    indices = np.argwhere(pre_filter == 255)
#     print(indices.shape)
    indices = indices[:,:2]
    disparity_filters = indices[:,0]
#     pp.pprint(disparity_filters)
    
    tmpI[np.isin(tmpI, disparity_filters)] = 0
    plot_image(pre_filter,8)
    plot_image(cv2.cvtColor(tmpI,cv2.COLOR_GRAY2BGR),9)
    
    print(" ============  Plotting Done !! =================")

In [444]:
plt.close('all')
flag_new_img = False
last_img = None
# umap = None
# vmap = None
# overlay = None
# helper_img = None
# filtered = None
# cpy1 = None
# cpy2 = None
# pre_filter = None

prev_e1,prev_e2,prev_greyThresh = 0, 0, 0
prev_map = False

found_imgs = grab_dir_images(homeDir)

In [445]:
# =========================================================
#
# =========================================================
p = interactive(line_finder,
    _img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ),
    method = {'Standard Hough Transform':0,'Probabilistic Hough Transform':1},
    line_input_method = {'Classic Thresholded':0,'Morphed Thresholded':1,'Canny Edges':2},
    e1 = (1,255,1), e2 = (1,255,1),
    ang = (0,360,1), rho = (1,1000,1),
    minLineLength = (0,255,1), maxLineGap = (0,255,1),
    houghThresh = (0,255,1), greyThresh = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = True, use_umap = True, flip_thresh_bin = False
)


row1 = HBox([p.children[0]])
row2 = HBox([p.children[1], p.children[2]], layout = Layout(flex_flow='row wrap'))
row3 = HBox([p.children[3], p.children[4]], layout = Layout(flex_flow='row wrap'))
row4 = HBox([p.children[5], p.children[6], p.children[7], p.children[8]], layout = Layout(display='flex',flex_flow='row'))
row5 = HBox([p.children[9], p.children[10], p.children[11]], layout = Layout(flex_flow='row wrap'))
row6 = HBox([p.children[12], p.children[13], p.children[14]], layout = Layout(flex_flow='row wrap'))
controls = VBox([row1,row2,row3,row4,row5,row6], layout = Layout(display='flex'))
output = p.children[-1]
display(VBox([controls, output]))

VkJveChjaGlsZHJlbj0oVkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oU2VsZWN0KGRlc2NyaXB0aW9uPXUnRmlsZTonLCBsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnMTAwJScpLCBvcHRpb27igKY=


In [59]:
print("""
Good Configuration:    for image = test/test_disparity.png)
---------------------------------------------------------------
    
    * Mapping Space       : U-Map
    * Line Finding Method : Probablistic Hough Transform
    * Filtering Used      : Composite Filtering -> Blurring
    * Kernel Size         : (2, 3)
    * Rho, Angle (deg)    :  1, 90
    * Min Line Length     :  11
    * Max Line Gap        :  22
    * Grey Thresholding   :  11
    * Hough Threshold     :  46
    * Contour Threshold   :  500
    
    * Raw Contour Areas        : [2762.0, 1751.0, 4845.5, 2999.0, 7708.5, 1574.0, 398.0])
    * Filtered Contour Areas   : [2762.0, 1751.0, 4845.5, 2999.0, 7708.5, 1574.0])
""")


Good Configuration:    for image = test/test_disparity.png)
---------------------------------------------------------------
    
    * Mapping Space       : U-Map
    * Line Finding Method : Probablistic Hough Transform
    * Filtering Used      : Composite Filtering -> Blurring
    * Kernel Size         : (2, 3)
    * Rho, Angle (deg)    :  1, 90
    * Min Line Length     :  11
    * Max Line Gap        :  22
    * Grey Thresholding   :  11
    * Hough Threshold     :  46
    * Contour Threshold   :  500
    
    * Raw Contour Areas        : [2762.0, 1751.0, 4845.5, 2999.0, 7708.5, 1574.0, 398.0])
    * Filtered Contour Areas   : [2762.0, 1751.0, 4845.5, 2999.0, 7708.5, 1574.0])



In [394]:
print(input_img.shape)
plot_image(cv2.cvtColor(input_img,cv2.COLOR_GRAY2BGR))
# bborder[np.where((bborder==[1,1,1]).all(axis=2))] = [255,0,255]

(480, 640)


In [493]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression,RANSACRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import (
    LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

ransac_img = cv2.imread("/home/hunter/devel/vision_playground/input-lines.png")
nonzero_pxls = ransac_img.nonzero()
nonzero_y = np.array(nonzero_pxls[0]).reshape(-1,1)
nonzero_x = np.array(nonzero_pxls[1]).reshape(-1,1)

# print(nonzero_x.shape, nonzero_y.shape)
# quadratic = PolynomialFeatures(degree=2)
# X_quad = quadratic.fit_transform(nonzero_x)
# print(X_quad)
# X_fit = np.arange(nonzero_x.min(), nonzero_x.max())[:, np.newaxis]

# ransac = RANSACRegressor()
# # model = make_pipeline(PolynomialFeatures(3), ransac)
# # ransac.fit(nonzero_x, -nonzero_y)

# model = make_pipeline(PolynomialFeatures(3), ransac)
# model.fit(nonzero_x, -nonzero_y)
# line_x = np.arange(nonzero_x.min(), nonzero_x.max())[:, np.newaxis]

x_plot = np.linspace(nonzero_x.min(), nonzero_x.max())
# y_plot = model.predict(x_plot[:, np.newaxis])
# y_plot = model.predict(line_x)
plt.close(5)
plt.figure(5)
# plt.scatter(nonzero_x, -nonzero_y, label='training points', color='lightgray')

z = np.polyfit(nonzero_x[:,0], -nonzero_y[:,0], 1)
p = np.poly1d(z)

print(z,p)
plt.scatter(nonzero_x, -nonzero_y)
plt.plot(x_plot, p(x_plot), 
         color='red', 
         lw=2,
         linestyle='-')
 
# plt.xlabel('% lower status of the population [LSTAT]')
# plt.ylabel('Price in $1000\'s [MEDV]')
# plt.legend(loc='upper right')
 
plt.tight_layout()
plt.show()

(array([   1.26229754, -340.1137798 ]), poly1d([   1.26229754, -340.1137798 ]))


In [470]:
# ransacL.fit(nonzerox_left, -nonzeroy_left)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Predict data of estimated models
line_x = np.arange(nonzero_x.min(), nonzero_x.max())[:, np.newaxis]
print(line_x.shape)
line_y = ransac.predict(line_x)



(179, 1)


ValueError: shapes (179,1) and (4,1) not aligned: 1 (dim 1) != 4 (dim 0)

In [462]:
plt.figure(5)
plt.scatter(nonzero_x, -nonzero_y, label='training points', color='lightgray')

plt.plot(line_x, -line_y, 
         color='red', 
         lw=2,
         linestyle='-')
 
plt.xlabel('% lower status of the population [LSTAT]')
plt.ylabel('Price in $1000\'s [MEDV]')
plt.legend(loc='upper right')
 
plt.tight_layout()
plt.show()

In [ ]:
# Robustly fit linear model with RANSAC algorithm
ransacL = linear_model.RANSACRegressor(residual_threshold=None, max_trials=50, stop_probability=0.90)
ransacL.fit(nonzerox_left, -nonzeroy_left)
inlier_maskL = ransacL.inlier_mask_
outlier_maskL = np.logical_not(inlier_maskL)

# Predict data of estimated models
# line_xL = np.arange(nonzerox_left.min(), nonzerox_left.max())[:, np.newaxis]
# line_yL = ransacL.predict(line_xL)

# cv2.line(display_lines,(int(line_xL[0]),int(-line_yL[0] + y_offset)),(int(line_xL[-1]),int(-line_yL[-1] + y_offset)),(255,0,0),thickness=3)

In [383]:
from sklearn.linear_model import (
    LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

nonzero_y = np.array(nonzero_pxls[0]).reshape(-1,1)
nonzero_x = np.array(nonzero_pxls[1]).reshape(-1,1)
ransac = RANSACRegressor()

estimators = [('OLS', LinearRegression()),
              ('Theil-Sen', TheilSenRegressor(random_state=42)),
              ('RANSAC', RANSACRegressor(random_state=42)),
              ('HuberRegressor', HuberRegressor())]
colors = {'OLS': 'turquoise', 'Theil-Sen': 'gold', 'RANSAC': 'lightgreen', 'HuberRegressor': 'black'}
linestyle = {'OLS': '-', 'Theil-Sen': '-.', 'RANSAC': '--', 'HuberRegressor': '--'}
lw = 3

x_plot = np.linspace(X.min(), X.max())
for title, this_X, this_y in [
        ('Modeling Errors Only', X, y),
        ('Corrupt X, Small Deviants', X_errors, y),
        ('Corrupt y, Small Deviants', X, y_errors),
        ('Corrupt X, Large Deviants', X_errors_large, y),
        ('Corrupt y, Large Deviants', X, y_errors_large)]:
    plt.figure(figsize=(5, 4))
    plt.plot(this_X[:, 0], this_y, 'b+')

    for name, estimator in estimators:
        model = make_pipeline(PolynomialFeatures(3), estimator)
        model.fit(this_X, this_y)
        mse = mean_squared_error(model.predict(X_test), y_test)
        y_plot = model.predict(x_plot[:, np.newaxis])
        plt.plot(x_plot, y_plot, color=colors[name], linestyle=linestyle[name],
                 linewidth=lw, label='%s: error = %.3f' % (name, mse))

    legend_title = 'Error of Mean\nAbsolute Deviation\nto Non-corrupt Data'
    legend = plt.legend(loc='upper right', frameon=False, title=legend_title,
                        prop=dict(size='x-small'))
    plt.xlim(-4, 10.2)
    plt.ylim(-2, 10.2)
    plt.title(title)
plt.show()

array([[2, 2],
       [4, 5]])
(array([0, 0]), array([0, 1]))
[[False False]
 [ True  True]]
array([[0, 0],
       [4, 5]])
array([[2, 2],
       [0, 0]])
